Let $s\in\{0,1\}^n$. (e.g. if $n=4$, then $s$ could be $1001$)

Define the function XOR$_s : \{0,1\}^n \rightarrow \{0,1\}$, which takes a n-bit string $x$ and returns XOR$_{s}(x) = \sum_{i=1}^{n} x_{i}s_{i}$ (mod 2)

Imagine someone gives you a chip with some XOR function on it but does not specify the $s$ from which it is constructed. Classically, you would need to evaluate the XOR function $n$ times to determine the value of $s$. With the power of quantum computing, you need only evaluate it once. 

In [3]:
"""Feel free to change the value of s. The quantum algorithm
should spit out whatever bitstring you input if you run this code block."""

import numpy as np
from cmath import polar

def XOR(s):
    """s is an n-bit string. XOR(s) is a unary function that performs 
    bitwise XOR between s and argument called x (formula shown above)"""
    def f(x):
        """x is another n-bit string"""
        output = 0
        for i in range(len(s)):
            output += int(s[i])*int(x[i])
        return output%2
    return f

"""This is the Hadamard gate, the most famous gate in quantum computing. 
It act on one qubit (represented by a 2 by 1 matrix) and produces one qubit (hence why it's a 2 by 2 matrix)"""
H = 0.5**0.5 * np.array([[1,1],\
                         [1,-1]])

"""The slogan of quantum computing is "Rotate, Compute, Rotate" (Ryan O'Donnell).
The "rotate" typically refers to some kind of Fourier transform. Below is the simplest
type of Fourier transform called the Hadamard Transform. It's useful when trying to detect 
XOR patterns, like we're doing here. To execute it, simply Hadamard every qubit you're working with. 

Note that this is not the type of transform used for Shor's Algorithm or Grover's Algorithm."""
def HFT(n):
    """n --> the number of qubits 
    Returns 2^n by 2^n Hadamard transform matrix"""
    if n == 1:
        return H
    return np.kron(HFT(n-1),H)
    """This last line uses the Kronecker product, also known as the tensor product, one
    of the most useful operations in all of quantum computing. Here's one of its main uses:
    say you want to apply gate A to your first qubit and gate B to your second qubit. Then, you can
    ask what gate you should apply to the composite system of both of your qubits to get the same result.
    The matrix of that composite gate is given by the tensor product of A and B."""

def plusminus(f, n):
    """n is some integer greater than or equal to 1
    f is a function on n-bit strings (i.e. it maps elements of {0,1}^n to {0,1})
    
    plusminus(f) is a matrix which sign-implements f compatibly with a quantum computer. 
    This means that it maps the column vector [0, 0, ... 0, 1, 0, ... ,0] where 1 is in the 
    ith position to (-1)**f(bit(i)) * [0, 0, 0, ... 0, 1, 0, ... ,0] where the 1 is again in the ith
    position and bit(i) is the bit representation of the number i
    
    Here is the key intuitive distinction to make: f can only act on n-bit strings. 
    It can't act on superpositions of n-bit strings. But plusminus(f,n), being a matrix, 
    has no problem acting on column vectors other than [0, 0, ... 0, 1, 0, ... ,0].
    In other words, it can act on superpositions by linear continuation."""
    N = 2**n
    #create list of bitstrings
    bitstrings = []
    for i in range(N):
        raw_string = format(i,"b")
        bitstrings.append('0'*(n-len(raw_string))+raw_string)
    #generate output matrix as specified
    output = np.zeros([N,N])
    for i in range(N):
        output[i,i] = (-1)**f(bitstrings[i])
    return output


def amp_to_prob(cx_number):
    """takes a complex number a+bi and returns its norm squared a^2+b^2
    
    this is how we get from an amplitude (the numbers stored in the column vector) 
    to an actual probability"""
    return polar(cx_number)[0]**2

def measure(state):
    """takes a state (column vector) and returns a bit-string, simulating a measurement of the state
    
    remember that a column vector [a_1, a_2, ..., a_N] has probabilty amp_to_prob(a_i) of resulting in 
    a measurement of |bit(i)> where bit(i) is the bitstring associated with the number i"""
    
    #this is fairly standard notation 
    #n --> number of qubits; N --> dimension of vectorspace we're dealing with
    N = len(state)
    n = int(np.log2(N))
    #create list of bitstrings
    bitstrings = []
    for i in range(N):
        raw_string = format(i,"b")
        bitstrings.append('0'*(n-len(raw_string))+raw_string)
    #generate random number from 0 to 1
    random = np.random.random()
    #generate pdf from quantum state
    pdf = list(map(amp_to_prob,state))
    #run through each element to determine measurement
    for i in range(len(pdf)):
        if random<pdf[i]:
            return bitstrings[i]
        random -= pdf[i]

"""Feel free to plug in any bit string s"""

s = "110101" #the mysterious s used to program the chip you were handed (you don't know this yet)

n = len(s)
N = 2**n
state = np.identity(N)[:,0] #initial quantum state |00...0>, represented by column vector [1, 0, 0, ..., 0]

#sequence of quantum gates we want to execute (notice the "rotate, compute rotate" pattern)
#since all of these gates act on all n qubits, there's no need to specify which wires we want to apply each gate to
sequence = [HFT(n),plusminus(XOR(s),n),HFT(n)] #notice how we only execute the XOR function once

#to perform the actual quantum computation, simply apply each gate (matrix) to the quantum state (vector)
for gate in sequence:
    state = np.dot(gate,state) #quantum computation

#final result, check to see if we indeed detected s
print('QUANTUM COMPUTER: "The value of s is {}."'.format(measure(state))) 

QUANTUM COMPUTER: "The value of s is 110101."
